# Scraper
Scrapes odds from unibet, predictions from fivethirtyeight and shows you the best bets.

In [38]:
# choose a league (0-5)
currentLeague = 1

# create the url matrix
import pandas as pd
urlUnibet = ['https://www.unibet.eu/betting#filter/football/netherlands/eredivisie',
              'https://www.unibet.eu/betting#filter/football/spain/laliga',
              'https://www.unibet.eu/betting#filter/football/germany/bundesliga',
              'https://www.unibet.eu/betting#filter/football/england/premier_league',
              'https://www.unibet.eu/betting#filter/football/france/ligue_1',
              'https://www.unibet.eu/betting#filter/football/italy/serie_a']
url538 = ['https://projects.fivethirtyeight.com/soccer-predictions/eredivisie/',
           'https://projects.fivethirtyeight.com/soccer-predictions/la-liga/',
           'https://projects.fivethirtyeight.com/soccer-predictions/bundesliga/',
           'https://projects.fivethirtyeight.com/soccer-predictions/premier-league/',
           'https://projects.fivethirtyeight.com/soccer-predictions/ligue-1/',
           'https://projects.fivethirtyeight.com/soccer-predictions/serie-a/']
dfUrl = pd.DataFrame(data={'urlUnibet': urlUnibet, 'url538': url538})
dfUrl

,url538,urlUnibet
0,https://projects.fivethirtyeight.com/soccer-pr...,https://www.unibet.eu/betting#filter/football/...
1,https://projects.fivethirtyeight.com/soccer-pr...,https://www.unibet.eu/betting#filter/football/...
2,https://projects.fivethirtyeight.com/soccer-pr...,https://www.unibet.eu/betting#filter/football/...
3,https://projects.fivethirtyeight.com/soccer-pr...,https://www.unibet.eu/betting#filter/football/...
4,https://projects.fivethirtyeight.com/soccer-pr...,https://www.unibet.eu/betting#filter/football/...
5,https://projects.fivethirtyeight.com/soccer-pr...,https://www.unibet.eu/betting#filter/football/...


## Odds from unibet

In [43]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep

In [45]:
# Start our headless (no GUI) browser
browser = webdriver.Firefox()

In [46]:
# Go to the URL
url = dfUrl.loc[currentLeague, 'urlUnibet']
browser.get(url)

# Wait until 1+ collapsible headers are found
try:
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_all_elements_located((By.Class, "KambiBC-collapsible-header"))
    )
finally:
    print('Done loading?')

# Click on them

    

# Get the source code
#soup = BeautifulSoup(browser.page_source, 'html5lib')

Done loading?


AttributeError: type object 'By' has no attribute 'Class'

In [41]:
element

NameError: name 'element' is not defined

In [32]:
# Find the dropdown bars
dropdowns = browser.find_elements_by_class_name('KambiBC-collapsible-header')

# Click on them
for dropdown in dropdowns:
    dropdown.click()
    sleep(1)

# Now get the source code
soup = BeautifulSoup(browser.page_source, 'html5lib')

In [33]:
# search for matches in the soup
matches = soup.find(id="KambiBC-container").find_all("li", class_="KambiBC-event-item")

# print amount found. Also remember this amount for getting the same amout from 538
numberOfMatches = len(matches)
print("Number of matches found: ", numberOfMatches)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [29]:
# quit the browser
browser.quit()

In [8]:
# Create empty list
jobs = []

for match in matches:
    # Create empty dictionary
    job = {}
    
    # Find info on the page
    #job["date"] = match.find(class_="KambiBC-event-item__start-time--date").text # <- Doesnt work..?
    teams = match.find_all(class_="KambiBC-event-participants__name")
    job["home_team"] = teams[0].text
    job["away_team"] = teams[1].text
    odds = match.find_all(class_="KambiBC-mod-outcome__odds")
    job["odd_home_win"] = odds[0].text
    job["odd_tie"] = odds[1].text
    job["odd_away_win"] = odds[2].text
    
    # Add to the list
    jobs.append(job)

In [9]:
# Make a data frame
dfOdds = pd.DataFrame(jobs)

# Show
#dfOdds

### Data transformation

In [10]:
# Change team names to lowercase
dfOdds["home_team"] = dfOdds["home_team"].str.lower()
dfOdds["away_team"] = dfOdds["away_team"].str.lower()

# Remove all accents
dfOdds["home_team"] = dfOdds["home_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')
dfOdds["away_team"] = dfOdds["away_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')

# Change the team names so that they match the ones in the 538 data frame
changes_nl = {'fc groningen': 'groningen',
              'fc twente': 'twente',
              'sc heerenveen': 'heerenveen',
              'fc utrecht': 'utrecht'}
changes_de = {'bayer leverkusen': 'leverkusen',
             'borussia monchengladbach': 'gladbach',
             'vfl wolfsburg': 'wolfsburg',
             'borussia dortmund': 'dortmund',
             'augsburg': 'fc ausburg'}
changes_es = {'deportiva las palmas': 'las palmas',
             'fc barcelona': 'barcelona'}
changes_en = {}
changes_fr = {'saint-etienne': 'st etienne',
             'paris sg': 'psg'}
changes_it = {'hellas verona': 'verona'}
changes = {**changes_nl, **changes_de, **changes_es, **changes_en, **changes_fr, **changes_it}
for old,new in changes.items():
    # Replace!
    dfOdds["home_team"] = dfOdds["home_team"].str.replace(old, new)
    dfOdds["away_team"] = dfOdds["away_team"].str.replace(old, new)

# Make 3-letter code names
dfOdds["home_code"] = dfOdds["home_team"].str[:3]
dfOdds["away_code"] = dfOdds["away_team"].str[:3]

# Convert to numbers
dfOdds["odd_home_win"] = pd.to_numeric(dfOdds["odd_home_win"])
dfOdds["odd_away_win"] = pd.to_numeric(dfOdds["odd_away_win"])
dfOdds["odd_tie"] = pd.to_numeric(dfOdds["odd_tie"])

# Show
dfOdds

KeyError: 'home_team'

## Predictions from five thirty eight

In [ ]:
from lxml import html
from lxml.cssselect import CSSSelector
import requests
import re
import pandas as pd

In [ ]:
# Go to the URL
url = dfUrl.loc[currentLeague, 'url538']
page = requests.get(url)

# Get the source code
tree = html.fromstring(page.content)

In [ ]:
# Get the same amount of matches as from unibet
matches = tree.cssselect('.games-container.upcoming .match-container')#[:numberOfMatches]
print("Number of matches found ", len(matches))

In [ ]:
cols = ['date', 'home_team', 'away_team', 'home_win', 'tie', 'away_win']
df538 = pd.DataFrame(columns=cols)

for idx in range(len(matches)):
    match = matches[idx]
    
    df538.at[idx, 'date'] = match.cssselect(".date div")[0].text
    df538.at[idx, 'home_team'] = match.cssselect(".match-top .name")[0].text
    df538.at[idx, 'home_win'] = match.cssselect(".match-top .prob")[0].text
    df538.at[idx, 'tie'] = match.cssselect(".tie-prob div")[0].text
    df538.at[idx, 'away_team'] = match.cssselect(".match-bottom .name")[0].text
    df538.at[idx, 'away_win'] = match.cssselect(".match-bottom .prob")[0].text
    
#df538

### Data transformation

In [ ]:
# Turn percentages into floats
df538['home_win'] = pd.to_numeric(df538['home_win'].str.replace("%",""))/100
df538['tie'] = pd.to_numeric(df538['tie'].str.replace("%",""))/100
df538['away_win'] = pd.to_numeric(df538['away_win'].str.replace("%",""))/100

# Drop the date column
#del df538["date"]

# Change team names to lowercase
df538["home_team"] = df538["home_team"].str.lower()
df538["away_team"] = df538["away_team"].str.lower()

# Remove all accents
df538["home_team"] = df538["home_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')
df538["away_team"] = df538["away_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')

# Make 3-letter code names
df538["home_code"] = df538["home_team"].str[:3]
df538["away_code"] = df538["away_team"].str[:3]

# SHow
df538

# Merge

In [ ]:
# check for missing matches for Unibet
dfCheck = df538.merge(dfOdds, how="outer", on=["home_code", "away_code"], indicator=True)
dfCheck[dfCheck._merge == 'right_only']

In [ ]:
# Merge by 3-letter codes
dfMerge = df538.merge(dfOdds, how="inner", on=["home_code", "away_code"])

# print
print('Number of matches succesfully merged:', len(dfMerge))

# Analyse

In [ ]:
# Calculate the expected values for win, tie and loss
dfMerge["expect_home"] = dfMerge["home_win"] * dfMerge["odd_home_win"]
dfMerge["expect_away"] = dfMerge["away_win"] * dfMerge["odd_away_win"]
dfMerge["expect_tie"] = dfMerge["tie"] * dfMerge["odd_tie"]

# Set threshold
threshold = 1.2

# Show
dfMerge.query('expect_home>@threshold or expect_away>@threshold or expect_tie>@threshold')\
    [["date", "home_team_x", "away_team_x", "expect_home", "expect_tie", "expect_away"]]